In [1]:
# imports
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader,TensorDataset, Dataset

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
from PIL import Image
import torchvision
from sklearn.model_selection import train_test_split


In [2]:
def img_loader(data_dir):
    filenames = [name for name in os.listdir(data_dir)]
    batch_size = len(filenames)
    out = torch.zeros(batch_size, 3, 256, 1024, dtype=torch.uint8)
    for i, filename in enumerate(filenames):
        out[i] = torchvision.io.read_image(os.path.join(data_dir, filename))
    return out

def find_folders_in_directory(directory):
    """
    Finds all folders in the given directory.

    :param directory: The directory to search for folders.
    :return: A list of paths to folders found in the given directory.
    """
    folders = []
    # Walk through the directory
    for entry in os.scandir(directory):
        # Check if the entry is a directory
        if entry.is_dir():
            folders.append(entry.path)
    return folders

# Example usage:
# Replace 'your_directory_path' with the path to the directory you want to search
def load_all_img(data_dir,folder_name):
    out = []
    fnames = find_folders_in_directory(data_dir)
    for fn in fnames:
        dir = os.path.join(fn,folder_name)
        dataf = img_loader(dir)
        if len(out) == 0:
            out = dataf
        else:
            out = torch.vstack((out, dataf))

    return out

In [3]:
input_img=load_all_img('/home/lambda2/carla_garage/data/ll_dataset_2023_05_10/Routes_Town04_ll_Repetition0', 'rgb')



In [4]:
input_img.shape

torch.Size([4582, 3, 256, 1024])

In [5]:
depth_img=load_all_img('/home/lambda2/carla_garage/data/ll_dataset_2023_05_10/Routes_Town04_ll_Repetition0', 'depth')

depth_img.shape

torch.Size([4582, 3, 256, 1024])

In [ ]:
plt.imshow(input_img[0][1])

In [ ]:

plt.imshow(depth_img[0][1])

In [ ]:
## dividing the dataset into test and train dataset
# test_train_ratio = 0.9
# pos_imgs_train = pos_imgs[0:int(np.floor(test_train_ratio*len(pos_imgs)))]
# pos_imgs_test = pos_imgs[int(np.floor(test_train_ratio*len(pos_imgs))):]


In [11]:


train_dataloader = DataLoader(dataset = TensorDataset(input_img, depth_img), batch_size=16, shuffle=True, num_workers=16)




In [12]:
torch.cuda.empty_cache()

In [13]:
class UNet(nn.Module):
    def __init__(self, c):
        super(UNet, self).__init__()
        # Convolutional Block in Encoder Section at Level i: ei   --   Decoder Section: di
        self.e01 = nn.Conv2d(in_channels= 3, out_channels=c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn01 = nn.BatchNorm2d(num_features=c)
        self.e02 = nn.Conv2d(in_channels=c, out_channels=c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn02 = nn.BatchNorm2d(num_features=c)
        
        self.e11 = nn.Conv2d(in_channels=c, out_channels=2*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn11 = nn.BatchNorm2d(num_features=2*c)
        self.e12 = nn.Conv2d(in_channels=2*c, out_channels=2*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn12 = nn.BatchNorm2d(num_features=2*c)
        
        self.e21 = nn.Conv2d(in_channels=2*c, out_channels=4*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn21 = nn.BatchNorm2d(num_features=4*c)
        self.e22 = nn.Conv2d(in_channels=4*c, out_channels=4*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn22 = nn.BatchNorm2d(num_features=4*c)
        
        self.e31 = nn.Conv2d(in_channels=4*c, out_channels=8*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn31 = nn.BatchNorm2d(num_features=8*c)
        self.e32 = nn.Conv2d(in_channels=8*c, out_channels=8*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn32 = nn.BatchNorm2d(num_features=8*c)
        
        
        self.d21 = nn.Conv2d(in_channels=8*c, out_channels=4*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn23 = nn.BatchNorm2d(num_features=4*c)
        self.d22 = nn.Conv2d(in_channels=4*c, out_channels=4*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn24 = nn.BatchNorm2d(num_features=4*c)
        
        self.d11 = nn.Conv2d(in_channels=4*c, out_channels=2*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn13 = nn.BatchNorm2d(num_features=2*c)
        self.d12 = nn.Conv2d(in_channels=2*c, out_channels=2*c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn14 = nn.BatchNorm2d(num_features=2*c)
        
        self.d01 = nn.Conv2d(in_channels=2*c, out_channels=c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn03 = nn.BatchNorm2d(num_features=c)
        self.d02 = nn.Conv2d(in_channels=c, out_channels=c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn04 = nn.BatchNorm2d(num_features=c)
        
        # Increaseing the resolution of image from a lower level to match the upper level
        self.upconv3 = nn.ConvTranspose2d(in_channels=8*c, out_channels=4*c, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(in_channels=4*c, out_channels=2*c, kernel_size=2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(in_channels=2*c, out_channels=c, kernel_size=2, stride=2)
        ### Fully connected layer
        self.fc1 = nn.Linear(40960, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 3)
        self.softmax = nn.Softmax(dim=3)

        # Output
        self.out = nn.Conv2d(in_channels=c, out_channels=3, kernel_size=3, stride=1, padding=1)
        
        self.maxpool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.sig=nn.Sigmoid()
        self.sm = nn.Softmax()

    def forward(self, x):
        # encoder
        ### x0: output encoder0 
        x0 = self.relu(self.bn02(self.e02(self.relu(self.bn01(self.e01(x))))))
        x = self.maxpool(x0)
        ### x1: output encoder 1
        x1 = self.relu(self.bn12(self.e12(self.relu(self.bn11(self.e11(x))))))
        x = self.maxpool(x1)
        
        ### x2: output encoder 2
        x2 = self.relu(self.bn22(self.e22(self.relu(self.bn21(self.e21(x))))))
        x = self.maxpool(x2)
        
        x = self.relu(self.bn32(self.e32(self.relu(self.bn31(self.e31(x))))))
        
        
        #### Classifier
        # xbn = x
        # xclass = xbn.reshape(xbn.size(0),-1)
        # xclass = self.relu(self.fc1(xclass))
        # xclass = self.relu(self.fc2(xclass))
        # xclass = self.relu(self.fc3(xclass))
        # xclass = (self.fc4(xclass))
        # xclass = self.softmax(xclass) 
        
        
        
        # out = x.reshape(x.size(0), -1)


        # decoder
        x = torch.cat((self.upconv3(x), x2), dim=1)
        x = self.relu(self.bn24(self.d22(self.relu(self.bn23(self.d21(x))))))
        
        x = torch.cat((self.upconv2(x), x1), dim=1)
        x = self.relu(self.bn14(self.d12(self.relu(self.bn13(self.d11(x))))))

        x = torch.cat((self.upconv1(x), x0), dim=1)
        x = self.relu(self.bn04(self.d02(self.relu(self.bn03(self.d01(x))))))

        out = self.out(x)
#         out=self.sig(out)
#         return out

        return  out

In [14]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.5):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
       
    def forward(self, pred_logits, target):
        pred = pred_logits.sigmoid()
        ce = F.binary_cross_entropy_with_logits(pred_logits, target, reduction = 'none')
        alpha = target*self.alpha + (1. - target )*(1. - self.alpha)
        pt = torch.where(target == 1, pred, 1- pred)
        return alpha* (1. - pt)**self.gamma * ce

In [15]:
#Initializing the network
device = 'cuda:3'
model = UNet(32).float().to(device)
model.train()

UNet(
  (e01): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn01): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (e02): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn02): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (e11): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (e12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (e21): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn21): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (e22): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn22): BatchNorm

In [16]:

n_epochs=15
learning_rate=0.001
# criterion1 = FocalLoss()
# criterion2 = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
# criterion1 = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)


In [24]:
data.shape, target.shape,output.shape

(torch.Size([1, 3, 256, 1024]),
 torch.Size([1, 3, 256, 1024]),
 torch.Size([1, 3, 256, 1024]))

In [25]:
data, target = train_dataloader.dataset[0:1]
output=model(data.float().to(device)).float().to(device)
loss = criterion(output, target)
#backward
optimizer.zero_grad()
loss.backward()
optimizer.step()
        
        # loss = criterion(output, target)

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 3; 23.68 GiB total capacity; 11.37 GiB already allocated; 9.62 MiB free; 11.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
#Train the network
loss_ls=[]
for epoch in range(n_epochs):
    loss_sum=0
    for batch_idx, (data, target) in enumerate(train_dataloader):


        data=data.float().to(device)
        target=target.float().to(device)



        #forward
        output=model(data).float().to(device)
        
        loss = criterion(output, target)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum=loss_sum+loss
        
    loss_ls.append(loss_sum)
    print('epoch:',epoch, '\n', loss_sum)
    PATH = 'm'+str(epoch)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_sum}, PATH)

    
    


RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 3; 23.68 GiB total capacity; 10.40 GiB already allocated; 145.62 MiB free; 11.27 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
loss_ls = torch.tensor(loss_ls)


In [ ]:
plt.plot(loss_ls)

In [ ]:
device

In [ ]:
# model = UNet(32).float()
# model.eval()
# PATH = 'm'+str(epoch)
# checkpoint = torch.load(PATH)
# # model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
#Test loss
loss_test_ls=[]
for epoch in range(n_epochs):
    model = UNet(32).float()
    PATH = 'm'+str(epoch)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    loss_sum=0
        
    

    for batch_idx, (data, target) in enumerate(test_dataloader):
        
        data=data.float().to(device)
       
        target=target.float().to(device)



        #forward
        output=model(data)
        out_decoder = output['out_decoder'].float().to(device)
        tclass = output['label'].float().to(device)
#         loss=criterion1(out_decoder, data)+criterion2(tclass, target)
#        loss1 = loss1.mean()

        loss = criterion1(tclass, target)+criterion2(out_decoder, data).mean()
        loss_sum=loss_sum+loss
#         loss = criterion2(out_decoder, data)
        #backward
        
    loss_test_ls.append(loss_sum)
    print(loss_sum)
    

    
    


In [ ]:
loss_test_ls = torch.tensor(loss_test_ls)

In [ ]:
plt.plot(loss_test_ls)

In [ ]:
train_loss_ls = []
for epoch in range(n_epochs):
    PATH = 'm'+str(epoch)
    checkpoint = torch.load(PATH)
    train_loss_ls.append(checkpoint[])

In [ ]:
output=model(data)
output['out_decoder'].min()

In [ ]:
criterion2(out_decoder, data).mean()

In [ ]:


test_adrs=8609
output=model((train_data[test_adrs:test_adrs+1,:,:,:]).to(device).float())
out_decoder = output['out_decoder'].cpu().detach()
out_decoder = torch.sigmoid(out_decoder)
# out_decoder = torch.abs(out_decoder)
# out_decoder = out_decoder/out_decoder.max()
tclass = output['label'].cpu().detach()
outplot = np.zeros([80,128,3])
outplot[:,:,0] = np.array(out_decoder[0][0])
outplot[:,:,1] = np.array(out_decoder[0][1])
outplot[:,:,2] = np.array(out_decoder[0][2])
img = (train_data[test_adrs])
inplot = np.zeros([80,128,3])
inplot[:,:,0] = np.array(img[0])
inplot[:,:,1] = np.array(img[1])
inplot[:,:,2] = np.array(img[2])
fig, ax = plt.subplots(2, sharex='row', sharey='row',figsize=(10,10))

im=ax[0].imshow(outplot)
# fig.colorbar(im,cax=ax[0])
im1=ax[1].imshow(inplot)
# fig.colorbar(im1,cax=ax[1])
print('predication:',F.sigmoid(tclass))
print('label:', label_train[test_adrs])

In [ ]:


test_adrs=95
output=model((test_data[test_adrs:test_adrs+1,:,:,:]).to(device).float())
out_decoder = output['out_decoder'].cpu().detach()
tclass = output['label'].cpu().detach()
outplot = np.zeros([80,128,3])
outplot[:,:,0] = np.array(out_decoder[0][0])
outplot[:,:,1] = np.array(out_decoder[0][1])
outplot[:,:,2] = np.array(out_decoder[0][2])
img = (test_data[test_adrs])
inplot = np.zeros([80,128,3])
inplot[:,:,0] = np.array(img[0])
inplot[:,:,1] = np.array(img[1])
inplot[:,:,2] = np.array(img[2])
fig, ax = plt.subplots(2, sharex='row', sharey='row',figsize=(10,10))

im=ax[0].imshow(outplot)
# fig.colorbar(im,cax=ax[0])
im1=ax[1].imshow(inplot)
# fig.colorbar(im1,cax=ax[1])
print('predication:',F.sigmoid(tclass))
print('label:', label_train[test_adrs])

In [ ]:
test_adrs=2
output=model(rs(train_data[test_adrs:test_adrs+1,:,:,:]).to(device).float())
out_decoder = output['out_decoder'].cpu().detach()
tclass = output['label'].cpu().detach()
outplot = np.zeros([80,128,3])
outplot[:,:,0] = np.array(out_decoder[0][0])
outplot[:,:,1] = np.array(out_decoder[0][1])
outplot[:,:,2] = np.array(out_decoder[0][2])

In [ ]:
out_decoder.min()

In [ ]:
plt.imshow(out_decoder[0][0])

In [ ]:
outplot = np.zeros([80,128,3])
outplot[:,:,0] = np.array(out_decoder[0][0])
outplot[:,:,1] = np.array(out_decoder[0][1])
outplot[:,:,2] = np.array(out_decoder[0][2])
plt.imshow(outplot)
# outplot = torch.tensor(outplot)

In [ ]:
####Metrics
# device = 'cuda:3'
# PATH = 'm'+str('14')
# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# model.to(device)
model.eval()

In [ ]:
def prd_dete(pred_label, number_class):
    #####Determining the pred_label
    l = np.eye(number_class)
    d = []
    for i in range(number_class):
        d.append(np.linalg.norm(np.array(pred_label.cpu().detach()) - l[i,:]))
         

    d = np.array(d)
    return np.argmin(d)



In [ ]:
prd_dete(torch.tensor([[0.99, 1, 0.3]]), 3)

In [ ]:
####Metrics
def conf_matrix(inp_tens):

    n_class = np.array([0,0,0])
    for it in range(len(inp_tens)):

        output=model((inp_tens[it:it+1,:,:,:]).to(device).float())
        out_decoder = output['out_decoder'].cpu().detach()
        out_decoder = torch.sigmoid(out_decoder)
        # out_decoder = torch.abs(out_decoder)
        # out_decoder = out_decoder/out_decoder.max()
        tclass = output['label'].cpu().detach()
        ncl = prd_dete(tclass,3)
        n_class[ncl] = n_class[ncl] +1
            
    return n_class

In [ ]:
# test_train_ratio = 0.9
# pos_imgs_train = pos_imgs[0:int(np.floor(test_train_ratio*len(pos_imgs)))]
# pos_imgs_test = pos_imgs[int(np.floor(test_train_ratio*len(pos_imgs))):]
# neg_imgs_train = neg_imgs[0:int(np.floor(test_train_ratio*len(neg_imgs)))]
# neg_imgs_test = neg_imgs[int(np.floor(test_train_ratio*len(neg_imgs))):]
# pos_imgs_train_label = torch.ones((len(pos_imgs_train),1))
# pos_imgs_test_label = torch.ones((len(pos_imgs_test),1))
# neg_imgs_train_label = torch.zeros((len(neg_imgs_train),1))
# neg_imgs_test_label = torch.zeros((len(neg_imgs_test),1))


In [ ]:
conf_matrix(pos_imgs_train)


In [ ]:
conf_matrix(neg_imgs_train)

In [ ]:
conf_matrix(noev_imgs_train)

In [ ]:
conf_matrix(pos_imgs_test)

In [ ]:
conf_matrix(neg_imgs_test)

In [ ]:
conf_matrix(noev_imgs_test)